In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "simulation"

## Prerequisites


### Example batch case

The following batch case is used as an example.


In [2]:
from models.case import BatchCase, Transaction

example_case = BatchCase(
    id="swap_usdt_usdc",
    description="Swap 100 USDT for USDC on Uniswap and transfer to Alice",
    steps=[
        Transaction(description="Approve USDT for Uniswap"),
        Transaction(description="Swap 100 USDT to USDC on Uniswap"),
        Transaction(description="Transfer USDC to Alice"),
    ],
)

### Contract addresses

Assume the following contract addresses are known.


In [3]:
# Contract addresses
USDT_ADDR = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
USDC_ADDR = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
UNISWAP_V2_ROUTER_ADDR = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"

# Dummy EOA
ALICE_ADDR = "0x436f795B64E23E6cE7792af4923A68AFD3967952"
BOB_ADDR = "0x8c575b178927fF9A70804B8b4F7622F7666bB360"

### Tools

We use web3.py to create the instances of the contracts.


In [4]:
import os
import json
from web3 import Web3

INFURA_API_KEY = os.getenv("INFURA_API_KEY")
web3 = Web3(Web3.HTTPProvider(f"https://mainnet.infura.io/v3/{INFURA_API_KEY}"))

with open("abi/erc20.json") as file:
    erc20_contract_json = json.load(file)

with open("abi/uniswap_v2_router.json") as file:
    uniswap_contract_json = json.load(file)

# Contract instances
usdc_contract = web3.eth.contract(abi=erc20_contract_json)
usdt_contract = web3.eth.contract(abi=erc20_contract_json)
uniswap_contract = web3.eth.contract(abi=uniswap_contract_json)

Example usage of the contract instances.


In [5]:
def get_balance(token_address: str, address: str):
    contract_instance = web3.eth.contract(
        address=token_address, abi=erc20_contract_json
    )
    encoded = contract_instance.encode_abi("balanceOf", args=[address])
    tx = {
        "to": token_address,
        "data": encoded,
    }

    try:
        # Perform the call
        result = web3.eth.call(tx)
        # Decode the result (balance) from bytes to integer
        decoded_result = int.from_bytes(result)
        # Convert to USDC (assuming 6 decimal places)
        usdc_balance = decoded_result / 10**6
        return usdc_balance
    except Exception as e:
        # Handle any errors that occur during the call
        print(f"An error occurred: {e}")


# Get balance of Bob
print("Bob's USDT balance:", get_balance(USDT_ADDR, BOB_ADDR))
print("Bob's USDC balance:", get_balance(USDC_ADDR, BOB_ADDR))

# Get balance of Alice
print("Alice's USDT balance:", get_balance(USDT_ADDR, ALICE_ADDR))
print("Alice's USDC balance:", get_balance(USDC_ADDR, ALICE_ADDR))

Bob's USDT balance: 0.0
Bob's USDC balance: 0.0
Alice's USDT balance: 4.0
Alice's USDC balance: 4.44435


## Simulate the transaction

Refer to the [Tenderly API documentation](https://docs.tenderly.co/node/rpc-reference/ethereum-mainnet/tenderly_simulateBundle) for more details.


In [6]:
import os
import requests
from typing import List

from langchain_core.pydantic_v1 import BaseModel, Field, validator


class TransactionParams(BaseModel):
    from_address: str = Field(description="The sender address")
    to_address: str = Field(description="The receiver address")
    data: str = Field(description="The data of the transaction")
    value: str = Field(description="The value of native token to send")

    @validator("from_address", "to_address")
    def check_address_format(cls, v):
        if not v.startswith("0x"):
            raise ValueError("Address must start with '0x'")
        if len(v) != 42:
            raise ValueError("Address must be 42 characters long")
        return v

    @validator("value")
    def check_value_format(cls, v):
        if not v.startswith("0x"):
            raise ValueError("Value must start with '0x'")
        return v

    @validator("data")
    def check_data_format(cls, v):
        if not v.startswith("0x"):
            raise ValueError("Data must start with '0x'")
        return v

In [7]:
def simulate_transaction(transactions: List[TransactionParams]) -> dict:
    """
    Simulate a transaction on Tenderly.

    Args:
        transactions (List[TransactionParams]): List of transactions to simulate.

    Returns:
        dict: The result of the simulation.
    """
    TENDERLY_API_KEY = os.getenv("TENDERLY_API_KEY")
    url = f"https://mainnet.gateway.tenderly.co/{TENDERLY_API_KEY}"
    data = {
        "id": 0,
        "jsonrpc": "2.0",
        "method": "tenderly_simulateBundle",
        "params": [
            [
                {
                    "from": tx.from_address,
                    "to": tx.to_address,
                    "data": tx.data,
                    "value": tx.value,
                }
                for tx in transactions
            ],
            "latest",
        ],
    }
    response = requests.post(url, json=data).json()
    if "error" in response:
        raise Exception(f"Simulation API error: {response['error']['message']}")
    if "result" in response:
        return response["result"]
    raise Exception(f"Unexpected response: {response}")

In [8]:
class AssetChange(BaseModel):
    contract_address: str = Field(description="The contract address of the asset")
    type: str = Field(description="The type of the asset change")
    name: str = Field(description="The name of the asset")
    symbol: str = Field(description="The symbol of the asset")
    decimals: int = Field(description="The decimals of the asset")
    raw_amount: str = Field(description="The raw amount in hex. Field 'rawAmount'")
    sender: str = Field(description="The sender of the transaction. Field 'from'")
    receiver: str = Field(description="The receiver of the transaction. Field 'to'")


class SimulationResult(BaseModel):
    asset_changes: List[AssetChange] = Field(
        description="List of assets changed during the transaction."
    )
    error: str = Field(
        description="Error message explaining why the transaction failed."
    )


def format_simulation_results(results: list[dict]) -> SimulationResult:
    """
    Processes a list of simulation results, extracting asset changes and any errors.

    Args:
        results (list[dict]): A list of dictionaries representing simulation results.

    Returns:
        SimulationResult: An object containing consolidated asset changes and any error messages encountered during the processing offailed results.

    Raises:
        Exception: If no results are provided, or if any result fails but no error message is found.
    """
    if not results:
        raise Exception("No results")

    all_asset_changes = []
    error = ""

    for result in results:
        # If a failed status is detected, attempt to extract the error immediately
        if not result["status"]:
            error = extract_error_from_trace(result.get("trace", []))
            if error:  # Break early if an error is found
                break

        # Only process asset changes if the result is successful
        asset_changes = [
            AssetChange(
                contract_address=change["assetInfo"]["contractAddress"],
                name=change["assetInfo"]["name"],
                symbol=change["assetInfo"]["symbol"],
                decimals=change["assetInfo"]["decimals"],
                type=change["type"],
                sender=change["from"],
                receiver=change["to"],
                raw_amount=change["rawAmount"],
            )
            for change in result.get("assetChanges", [])
        ]
        all_asset_changes.extend(asset_changes)

    # Check for any failure after all results are processed
    if any(result["status"] is False for result in results) and not error:
        raise Exception("No error message")

    return SimulationResult(asset_changes=all_asset_changes, error=error)


def extract_error_from_trace(trace_list: list[dict]) -> str:
    """Extract error and error reason from trace list if applicable."""
    for trace in reversed(trace_list):
        error = trace.get("error", "")
        error_reason = trace.get("errorReason", "")
        if error:
            return f"{error}: {error_reason}" if error_reason else error
    return ""

In [9]:
def pretty_print(result: SimulationResult):
    """
    Print the result of the simulation in a readable format.
    """
    print(f"The transaction was {'failed. 😕' if result.error else 'successful! 🎉'}")

    if result.error:
        print("-------------")
        print(result.error)

    alice_address = ALICE_ADDR.lower()
    for asset_change in result.asset_changes:
        outgoing_sign = "-" if asset_change.sender.lower() == alice_address else "+"
        amount = int(asset_change.raw_amount, 16)
        symbol = asset_change.symbol.upper()

        print(
            f"-------------\n"
            f"{symbol} ({asset_change.contract_address})\n"
            f"Change: {outgoing_sign}{amount:,} ({asset_change.raw_amount})"
        )

## Example - Successful


In [10]:
import time

example_tx = [
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=USDC_ADDR,
        data=usdc_contract.encode_abi(
            "approve", args=[UNISWAP_V2_ROUTER_ADDR, 1_000_000]
        ),
        value="0x0",
    ),
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=UNISWAP_V2_ROUTER_ADDR,
        data=uniswap_contract.encode_abi(
            "swapExactTokensForTokens",
            args=[
                1_000_000,
                0,
                [USDC_ADDR, USDT_ADDR],
                ALICE_ADDR,
                int(time.time()) + 500,
            ],
        ),
        value="0x0",
    ),
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=USDT_ADDR,
        data=usdc_contract.encode_abi("transfer", args=[BOB_ADDR, 4_000_001]),
        value="0x0",
    ),
]

tx_results = format_simulation_results(simulate_transaction(example_tx))
print(tx_results)

asset_changes=[AssetChange(contract_address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', type='Transfer', name='USDC', symbol='usdc', decimals=6, raw_amount='0xf4240', sender='0x436f795b64e23e6ce7792af4923a68afd3967952', receiver='0x3041cbd36888becc7bbcbc0045e3b1f144466f5f'), AssetChange(contract_address='0xdac17f958d2ee523a2206206994597c13d831ec7', type='Transfer', name='Tether', symbol='usdt', decimals=6, raw_amount='0xf41f2', sender='0x3041cbd36888becc7bbcbc0045e3b1f144466f5f', receiver='0x436f795b64e23e6ce7792af4923a68afd3967952'), AssetChange(contract_address='0xdac17f958d2ee523a2206206994597c13d831ec7', type='Transfer', name='Tether', symbol='usdt', decimals=6, raw_amount='0x3d0901', sender='0x436f795b64e23e6ce7792af4923a68afd3967952', receiver='0x8c575b178927ff9a70804b8b4f7622f7666bb360')] error=''


In [11]:
pretty_print(tx_results)

The transaction was successful! 🎉
-------------
USDC (0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48)
Change: -1,000,000 (0xf4240)
-------------
USDT (0xdac17f958d2ee523a2206206994597c13d831ec7)
Change: +999,922 (0xf41f2)
-------------
USDT (0xdac17f958d2ee523a2206206994597c13d831ec7)
Change: -4,000,001 (0x3d0901)


## Example - Failed


In [12]:
import time

example_tx = [
    TransactionParams(
        from_address=ALICE_ADDR,
        to_address=UNISWAP_V2_ROUTER_ADDR,
        data=uniswap_contract.encode_abi(
            "swapExactTokensForTokens",
            args=[
                1_000_000,
                0,
                [USDC_ADDR, USDT_ADDR],
                ALICE_ADDR,
                int(time.time()) + 500,
            ],
        ),
        value="0x0",
    ),
]

tx_results = format_simulation_results(simulate_transaction(example_tx))
print(tx_results)

asset_changes=[] error='execution reverted: ERC20: transfer amount exceeds allowance'


In [13]:
pretty_print(tx_results)

The transaction was failed. 😕
-------------
execution reverted: ERC20: transfer amount exceeds allowance


In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
response = llm.invoke(f"Summarize the following transaction: {tx_results}")
print(response.content)

The transaction failed due to an error indicating that the transfer amount exceeded the allowed limit. No changes were made to the assets as a result of this failed transaction.
